In [1]:
from fastapi import FastAPI, UploadFile
from PIL import Image

In [2]:
app = FastAPI()

@app.get("/")
def hello():
    return {"message": "hello"}

@app.post("/upload")
async def create_upload_file(file: UploadFile):
    return {"filename": file.filename}

In [5]:
app.run()

AttributeError: 'FastAPI' object has no attribute 'run'

In [1]:
from ultralytics import YOLO
from PIL import Image
import os
import sys
import random

In [2]:
model_type = "segment"

dataset = ""

if model_type=="segment":
    dataset = "./datasets/car-damage-detection-3/data.yaml"
else:
    dataset = "./datasets/Object-Detection-Car-Damage-1/data.yaml"

In [3]:
model_list = ["best medium 8 classes.pt", 
              "detect best 3 classes.pt", 
              "object detect 60 epoch.pt", 
              "object detect 70 epoch.pt", 
              "object detection 80 epochs.pt", 
              "object detect 95 epochs.pt"]

In [7]:
model_name = model_list[1]

In [8]:
model = YOLO(f"./models/{model_name}")

In [12]:
image_names = os.listdir("./Car damage detection.v3i.yolov8/valid/images")
random.shuffle(image_names)
image_names = image_names[:30]

In [14]:
image_paths = ["./Car damage detection.v3i.yolov8/valid/images/"+image_name for image_name in image_names]

In [6]:
images = [Image.open(image_path) for image_path in image_paths]

In [7]:
results = model.predict(images)


0: 640x640 1 Light, 1: 640x640 1 Door, 2: 640x640 1 Bumper, 1 Dickey, 3: 640x640 1 Fender, 1 Light, 4: 640x640 (no detections), 5: 640x640 1 Fender, 6: 640x640 1 Door, 7: 640x640 2 Doors, 8: 640x640 1 Bumper, 1 Fender, 9: 640x640 1 Bumper, 10: 640x640 1 Windshield, 11: 640x640 1 Door, 1 Light, 12: 640x640 (no detections), 13: 640x640 1 Bumper, 14: 640x640 1 Light, 15: 640x640 1 Bonnet, 1 Bumper, 2 Lights, 16: 640x640 1 Bumper, 17: 640x640 (no detections), 18: 640x640 1 Fender, 19: 640x640 2 Doors, 1 Light, 20: 640x640 1 Door, 1 Fender, 21: 640x640 (no detections), 22: 640x640 1 Bumper, 1 Fender, 1 Light, 23: 640x640 1 Bumper, 24: 640x640 1 Fender, 25: 640x640 1 Fender, 26: 640x640 1 Light, 27: 640x640 1 Door, 1 Fender, 28: 640x640 1 Bumper, 29: 640x640 (no detections), 1587.0ms
Speed: 2.0ms preprocess, 52.9ms inference, 12.6ms postprocess per image at shape (1, 3, 640, 640)


In [11]:
os.mkdir(f"./Results/{model_name}")

In [15]:
path = f"./Results/{model_name}"

In [16]:
def save_images(result, image_name, path):
    im_array = result.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.save(f'{path}/{image_name}')


In [17]:
for i in range(len(results)):
    save_images(results[i], image_names[i], path)

In [18]:

from roboflow import Roboflow
rf = Roboflow(api_key="4CYtMECVZIaiVUXX1LAx")
project = rf.workspace("trialworkspace-n8ail").project("object-detection-car-damage")
dataset = project.version(1).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.237, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Object-Detection-Car-Damage-1 in yolov8:: 100%


In [19]:

from roboflow import Roboflow
rf = Roboflow(api_key="4CYtMECVZIaiVUXX1LAx")
project = rf.workspace("trialworkspace-n8ail").project("car-damage-detection-hndoe")
dataset = project.version(3).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.237, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to car-damage-detection-3 in yolov8:: 100%|█| 117


In [30]:
dataset

'./datasets/car-damage-detection-3/data.yaml'

In [32]:
vals = model.val(data=dataset)

Ultralytics YOLOv8.0.237 🚀 Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)


val: Scanning /home/parth/Desktop/hackvita/datasets/car-damage-detection-3/valid
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         48         93     0.0322     0.0961     0.0283    0.00974     0.0197     0.0587      0.011    0.00318
                Bonnet         48         41     0.0551      0.171     0.0612     0.0193     0.0315     0.0976     0.0179    0.00358
                Bumper         48          1          0          0          0          0          0          0          0          0
                Dickey         48         51     0.0414      0.118     0.0238    0.00997     0.0276     0.0784      0.015    0.00596
Speed: 4.2ms preprocess, 58.4ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/segment/val4


In [34]:
vals.maps

array([    0.02285,           0,    0.015921,    0.012924,    0.012924,    0.012924,    0.012924])

In [39]:
with open("testfile", "a") as file:
    file.write(model_name+"\n")
    file.write("maps= "+str(vals.maps)+"\n")
    file.write("confusion_matrix= "+str(vals.confusion_matrix)+"\n")
    file.write("f1_score= "+str(vals.box.f1)+"\n")

In [51]:
vals.box.f1.shape

(3,)

In [9]:
model.task

'detect'